In [4]:
import os
import csv
import json
#import pandas as pd
from openai import OpenAI

# ENV
# Set API key
API_KEY = ""
# System prompt
SYSTEM_PROMPT = "You are a helpful assistant that collects data from emails."

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=API_KEY)
#client = OpenAI(api_key=userdata.get('openai'))

In [8]:
# Folder containing email HTML files
#input_folder = "/content/drive/My Drive/20_UNI/TUM/2425_DataVizR/bahnmails"
input_folder = "/test"
#output_csv = "/content/drive/My Drive/20_UNI/TUM/2425_DataVizR/bahnmails/email_data.csv"
output_csv = "/test/email_data.csv"

In [ ]:
# Function to read an HTML file
def read_html_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read() # put html into var "content"
    return content
    #return file.read() # same output

html_content = read_html_file('test/mailexp_20250114-_noreply@deutschebahn.com_-1323.html')
#print(html_content)

In [11]:
# Function to process HTML content with AI
def extract_email_info(html_content):
    prompt = """Here is a booking confirmation from Deutsche Bahn. 
    Please arrange the following key information in a comma-separated table 
    with the header: date, departure_time, departure_station, arrival_time, 
    arrival_station, price, booking_timestamp, order_number"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": html_content}
        ],
        temperature=0.1,  # Set to 0 for a deterministic answer
        max_tokens=2000   # adjust
    )
    
    return response.choices[0].message.content.strip()


In [12]:
extract_email_info(html_content)

'```plaintext\ndate, departure_time, departure_station, arrival_time, arrival_station, price, booking_timestamp, order_number\n01/02/2025, 18:51, München Hbf, 22:52, Berlin Hbf, 18.74 EUR, 14/01/2025 22:08, 982732871065\n```'

In [14]:
# Main Execution
email_data = []

extracted_text = extract_email_info(html_content)

# Process AI response (Assuming CSV-like output)
lines = extracted_text.split("\n")
for line in lines:
    parts = line.split(",")  # Assuming AI returns comma-separated values
    if len(parts) == 2:
        email_data.append(parts)

# Save extracted data to CSV
save_to_csv(email_data, output_csv)

In [7]:
# Function to save extracted data to CSV
def save_to_csv(data, output_file):
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["date", "departure_time", "departure_station", 
                         "arrival_time", "arrival_station", "price", 
                         "booking_timestamp", "order_number"])  # CSV Headers

        for row in data:
            writer.writerow(row)

In [ ]:
# Main Execution
email_data = []

# Process each HTML email file
for filename in os.listdir(input_folder):
    if filename.endswith(".html"):
        file_path = os.path.join(input_folder, filename)
        html_content = read_html_file(file_path)
        extracted_text = extract_email_info(html_content)

        # Process AI response (Assuming CSV-like output)
        lines = extracted_text.split("\n")
        for line in lines:
            parts = line.split(",")  # Assuming AI returns comma-separated values
            if len(parts) == 2:
                email_data.append(parts)

# Save extracted data to CSV
save_to_csv(email_data, output_csv)

print(f"CSV saved to {output_csv}")

# Print the model's reply
#print("Response:", response.choices[0].message.content.strip())

# Print token usage details
#print("Tokens Used:", response.usage)